# Spotify Music Recommendation Model

> Modeling recommendation relationships between spotify tracks using the Million Playlist dataset and graph theory.

## Links

<ul>
    <li><a href="https://www.kaggle.com/datasets/himanshuwagh/spotify-million">Million Playlist Dataset</a></li>
</ul>

## Importing Modules and Loading Data

This model will be using NumPy to perform fast, vectorized arithmetic on an adjacency matrix. The JSON and os libraries will also be used to read the .json files. 

Loading the data is straightforward. A few maps are created that map the uri (unique identifier of a spotify track) to other data such as the track's name and index in the adjacency matrix. All the tracks are stored in a NumPy 2D array, where each row is a playlist.

In [8]:
import numpy as np
import os
import json

In [9]:
%%time
# loading data

files = os.listdir('./data')

uriToIdx = {}
idxToURI = {}
uriToName = {}
allPlaylists = np.full((12_000, 70), -1, dtype='int32')
counter = 0

for file in files:
    if file == '.DS_Store':
        continue
    with open(f"./data/{file}") as f:
        d = json.load(f)
        playlists = d["playlists"]
        for i in range(len(playlists)):
            playlist = playlists[i]["tracks"]
            for j in range(len(playlist)):
                if playlist[j]["track_uri"] not in uriToIdx:
                    uriToIdx[playlist[j]["track_uri"]] = len(uriToIdx)
                    idxToURI[uriToIdx[playlist[j]["track_uri"]]] = playlist[j]["track_uri"]
                    uriToName[playlist[j]["track_uri"]] = playlist[j]["track_name"]



for file in files:
    if file == '.DS_Store':
        continue
    with open(f"./data/{file}") as f:
        d = json.load(f)
        playlists = d["playlists"]
        for i in range(len(playlists)):
            playlist = playlists[i]["tracks"]
            for j in range(len(playlist)):
                if j >= 70:
                    break
                allPlaylists[i + counter, j] = uriToIdx[playlist[j]["track_uri"]]
    counter += 1000

CPU times: user 3.3 s, sys: 190 ms, total: 3.49 s
Wall time: 3.52 s


## Representing Music Recommendations as a Graph

A track is "connected" to another track if they are in the same playlist. These connections are weighted. For example, if a track and some other track are both in 20 playlists, their connection has a strong weight. On the other hand, if a track and some other track are both in only 1 playlist, their connection is weak. Thus, the relationships between all spotify tracks can be represented as a <b>connected and undirected graph</b>, where the tracks are nodes. The neighbors of a node with the strongest connections are therefore the songs that should be recommended.

Intuitively, this makes sense. If you like a song and 20 other playlists have that song with another song, then that other song should be recommended.

## Design Decisions and Memory Complexity...

Only 12,000 playlists will be used to populate the adjacency matrix. This is because using the adjacency matrix requires $O(n^2)$ memory complexity, where $n$ is the number of nodes (tracks). Therefore, with ~180,000 tracks, a lot of RAM will be needed. Google CoLab could be used to buy more RAM, but that costs money.

Here is why the alternative graph representations weren't used:
<ul>
    <li><b>Sparse Adjacency Matrix</b> - the graph of tracks isn't sparse. Each track has ~350 neighbors, so creating a sparse matrix wouldn't save much memory and the time compexity costs of populating the sparse matrix would far outweight the memory benefits.</li>
    <li><b>Adjacency List</b> - creating variable-length lists would mitigate the memory complexity, but would be outweighed by the time complexity costs, as NumPy does not support variable-length data structures. Using Python's built-in lists would take hours to days.</li>
</ul>

## Implementation

The adjacency matrix is a simple $n \times n$ NumPy array, with a structured datatype that stores the "weight" of each connection. Fast, vectorized NumPy operations are performed for each playlist to populate the adjacecny matrix with tracks and weights.

It takes less than a minute to load ~180,000 tracks from 12,000 playlists into the adjacency matrix. Not bad NumPy!

In [10]:
%%time
# populate adjacency matrix

n = len(uriToIdx)
dt = [("track", np.uint32), ("weight", np.uint32)]
adj_matrix = np.zeros((n, n), dtype=dt)

view = adj_matrix.view(dtype=np.int32)

for i in range(12_000):
    filtered = allPlaylists[i][allPlaylists[i] != -1]
    arr = np.array(np.meshgrid(filtered, filtered)).T.reshape(-1, 2)
    arr = arr[arr[:, 0] != arr[:, 1]]
    view[arr[:, 0], arr[:, 1]] = 1
    view[arr[:, 0], (arr[:, 1] * 2) + 1] += 1

CPU times: user 4.53 s, sys: 25.7 s, total: 30.3 s
Wall time: 46.5 s


# Creating an API for our Model

Creating a few functions to easily get recommendations from our adjacency matrix and print recommendations to the console.

In [11]:
# functions for interfacing with adjacency matrix and maps

def get_uri(url):
    parts = url.split('/')
    return parts[len(parts) - 1]

def add_prefix(uri):
    return f"spotify:track:{uri}"

def get_recommendations(url, numRecs=5):
    try:
        trackIdx = uriToIdx[add_prefix(get_uri(url))]
    except KeyError:
        print("song not found in database")
        return None
    result = []
    track = adj_matrix[trackIdx, :]
    recommendations = track["weight"].argsort()[:-(numRecs + 1):-1]
    for recTrackIdx in recommendations:
        result.append((uriToName[idxToURI[recTrackIdx]], adj_matrix[trackIdx, recTrackIdx]["weight"]))
    return result

def print_recommendations(recs):
    if recs is None:
        print("no recommendations")
    for recSong, numAprs in recs:
        print(f"{recSong} - {numAprs}")

## Model Performance

Getting track recommendations has $O(1)$ time complexity, as we are simply "looking up" the track in the adjacency matrix and finding the elements with the highest weights.

It takes less than 10ms to retrieve 10 recommendations for a sample track.

### Recommendation Quality

The "quality" of track recommendations is subjective. The tracks recommended for this Fluorescent Adolescent by Arctic Monkeys have some clear similarities with the track. For example, "Do I Wanna Know?" and "Why'd You Only Call Me When You're High?" are also by Arctic Monkeys. Additionally, "Come a Littler Closer", "1901", "Someday" and "Take Me Out" are also of the alternative rock genre. The other tracks are related in more subtle ways.

In [12]:
%%time

# Fluorescent Adolescent by Arctic Monkeys
recommendations = get_recommendations("https://open.spotify.com/track/7e8utCy2JlSB8dRHKi49xM", numRecs=10)

CPU times: user 2.28 ms, sys: 1.55 ms, total: 3.82 ms
Wall time: 2.63 ms


In [13]:
print_recommendations(recommendations)

Do I Wanna Know? - 14
Come a Little Closer - 14
1901 - 10
Take Me Out - 9
A-Punk - 9
Tighten Up - 8
Why'd You Only Call Me When You're High? - 8
Someday - 8
What You Know - 8
Breezeblocks - 8


## Potential Improvements

While the recommendations are already pretty good, they can be further improved by preprocessing the million total playlists to find a diverse set of ideal "recommendation" playlists.

Next, 12,000 playlists and ~180,000 songs leaves out millions of songs that are available on Spotify. The current limitation is the memory complexity of our adjacency matrix. A potential solution would be using a different graph implementation that uses less mmemory (sparse matrix, adjacency list, etc.).

Finally, exporting the model is difficult because the adjacency matrix is so large (it takes up gigabytes of memory), which means makes it hard to deploy it on a web application. A potential solution would be using another graph implementation